## 4. <a id='4_cell'></a>Create Group using Excel, Pandas and the PyFHIR models....

This uses Pandas to easily upload patient data from an Excel (or CSV) file based
on some tricks I learned to create instancea fo resources a lot more complex than patient such as conformance
resources like CapabilityStatement and StructureDefinitition.

- create group resources for combinations of:

1. all patients
1. all patients + timeframe ( today, tomorrow )
1. all patients + status
1. practitioner
1. practitioner + timeframe ( today, tomorrow )
1. practitioner + status
1. location
1. location + timeframe ( today, tomorrow )
1. location + status
1. service
1. service + timeframe ( today, tomorrow )
1. service + status

In [20]:
parameters = ['service']    #['all','practitioner','active','discharged','location','service']
timeframe = 'today' #[None,'today','tomorrow']

- Then validate and load to ref server
- Then perform searches and print out results - ( print out transactions as markdown )

- Todo using Synthea generated patients and providers


### Load and excel file using Pandas

I've made a sample excel file `my_patients.xlsx` containing data for several patients.
Note that you will need to install the *pandas* module.

- See this set of tutorials to learn more about Pandas:

   [![pandas_tut](pandas.png)](https://www.youtube.com/watch?v=ZyhVh-qRZPA&list=PL-osiE80TeTsWmV9i9c58mdDCSskIFdDS "Pandas Tutorial")

In [21]:
cd ~/pyFHIR_models # This is to make sure in right directory for Binder

[Errno 2] No such file or directory: '/Users/ehaas/pyFHIR_models # This is to make sure in right directory for Binder'
/Users/ehaas/Documents/Python/MyBinder/pyFHIR_models


In [22]:
from pandas import *
df = read_excel('my_patients.xlsx','Sheet1',na_filter = False, dtype=str)
# treat all empty cells as '' and convert all data including booleans to string  

df

,row,my_id,f_name,l_name,gender,b_date,is_deceased,is_active,phone,address1,city,state,zip,practitioner,location,service,admit_date,discharge_date
0,1,123,Amy,Brown,F,1964-06-19 00:00:00,False,True,5555555555,100 Main St,Napa,CA,94559,smith,Queen,proctology,2020-05-02 00:00:00,2020-05-03 00:00:00
1,2,234,Bert,Black,M,1969-08-09 00:00:00,False,True,5555555555,100 Oak St,Napa,CA,94558,jones,St Helena,urology,2020-05-02 00:00:00,2020-05-03 00:00:00
2,3,345,Colleen,Blue,F,2000-09-21 00:00:00,False,False,5555555555,100 Spruce St,Napa,CA,94559,smith,Queen,dermatology,2020-05-02 00:00:00,2020-05-04 00:00:00
3,4,456,Daniel,Crimson,M,1996-05-23 00:00:00,False,True,5555555555,100 Yahome St,Napa,CA,94558,jones,St Helena,proctology,2020-05-03 00:00:00,2020-05-04 00:00:00
4,5,567,Elizabeth,Green,F,1941-10-21 00:00:00,False,True,5555555555,100 Trancas Blvd,Napa,CA,94559,smith,Queen,urology,2020-05-03 00:00:00,2020-05-04 00:00:00
5,6,689,Frederick,Gray,M,1964-06-26 00:00:00,False,True,5555555555,100 A St,Napa,CA,94558,jones,St Helena,dermatology,2020-05-03 00:00:00,2020-05-04 00:00:00
6,7,890,Greg,Orange,M,1982-05-01 00:00:00,False,True,5555555555,100 Elm St,Napa,CA,94558,smith,Queen,proctology,2020-05-03 00:00:00,2020-05-05 00:00:00
7,8,901,Harriet,Silver,F,2002-02-02 00:00:00,False,True,5555555555,100 E St,Napa,CA,94558,jones,St Helena,urology,2020-05-04 00:00:00,2020-05-06 00:00:00
8,9,902,Imelda,Gold,F,1959-03-19 00:00:00,False,True,5555555555,100 Imola St,Napa,CA,94558,smith,Queen,dermatology,2020-05-04 00:00:00,2020-05-05 00:00:00
9,10,903,Jack,White,M,1988-08-08 00:00:00,False,True,5555555555,200 Trancas Blvd,Napa,CA,94558,jones,St Helena,proctology,2020-05-04 00:00:00,2020-05-05 00:00:00


In [23]:
df.dtypes

row               object
my_id             object
f_name            object
l_name            object
gender            object
b_date            object
is_deceased       object
is_active         object
phone             object
address1          object
city              object
state             object
zip               object
practitioner      object
location          object
service           object
admit_date        object
discharge_date    object
dtype: object

#### convert admit-date to datetime object

In [24]:
#df['admit_date'] = to_datetime(df['admit_date'])
#df['discharge_date'] = to_datetime(df['discharge_date'])

#### getting min mas and unique values for row

In [25]:
df.admit_date.max(), df.admit_date.min(), getattr(df, "location").unique(), df.practitioner.unique(), df.service.unique()

('2020-05-04 00:00:00',
 '2020-05-02 00:00:00',
 array(['Queen', 'St Helena'], dtype=object),
 array(['smith', 'jones'], dtype=object),
 array(['proctology', 'urology', 'dermatology'], dtype=object))

### How to Access Dataframe Data Using dot.notation

To do this use the [itertuples method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html).

In [26]:
for i in df.itertuples(index=True):     
    print(f'i.row = {i.row}: i.f_name = {i.f_name}, i.l_name = {i.l_name}, i.gender = {i.gender} , etc...')

i.row = 1: i.f_name = Amy, i.l_name = Brown, i.gender = F , etc...
i.row = 2: i.f_name = Bert, i.l_name = Black, i.gender = M , etc...
i.row = 3: i.f_name = Colleen, i.l_name = Blue, i.gender = F , etc...
i.row = 4: i.f_name = Daniel, i.l_name = Crimson, i.gender = M , etc...
i.row = 5: i.f_name = Elizabeth, i.l_name = Green, i.gender = F , etc...
i.row = 6: i.f_name = Frederick, i.l_name = Gray, i.gender = M , etc...
i.row = 7: i.f_name = Greg, i.l_name = Orange, i.gender = M , etc...
i.row = 8: i.f_name = Harriet, i.l_name = Silver, i.gender = F , etc...
i.row = 9: i.f_name = Imelda, i.l_name = Gold, i.gender = F , etc...
i.row = 10: i.f_name = Jack, i.l_name = White, i.gender = M , etc...


### Create Group Instance for All Patients, One member for Each Row...

1. initialize group resource and datatype instances
1. iterate over dataframe rows
1. assign dataframe data to member elements (may need to massage the strings to fit the data-types - see `active` for example)
1. validate and save the resources in the reference server
1. option to create narrative using jinja2 templates (that is a topic for another notebook)

### import all the modules you need

In [27]:
from fhir_model_generator.model import fhirdate, fhirreference, group, patient, identifier, humanname, contactpoint, address
from json import dumps
from requests import get, post
from datetime import datetime, date, timedelta
from IPython.display import display as Display, HTML, Markdown

### Initialize the Group Resource

In [28]:
pl_use_case = 'all_patients'
my_g = group.Group(
        dict(
            type = 'person',
            actual = True,
            name = pl_use_case,
            active = True,
            #quantity = df.shape[0],          
        )
) # initialize group instance

print(dumps(my_g.as_json(), indent =4))

{
    "resourceType": "Group",
    "active": true,
    "type": "person",
    "actual": true,
    "name": "all_patients"
}


### add an identifier and the managingEntity element - i.e. the Organization that created the group.

In [29]:
my_id = identifier.Identifier( # add complex identifier element
    dict(
        value = f'group-{str(datetime.now().isoformat())}',
        system = "http://example.org",
        ))
my_g.identifier = [my_id]

my_me = fhirreference.FHIRReference(
    dict(
        #reference = "http://example.org",
        display = "example managingEntity",
    )
)
my_g.managingEntity = my_me

print(dumps(my_g.as_json(), indent =4))

{
    "resourceType": "Group",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-2020-05-03T14:41:49.412645"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "all_patients",
    "managingEntity": {
        "display": "example managingEntity"
    }
}


### Add Group Characteristics

- define the parameters to filter by  here based on the use case
- for All patients - no filters
- create a function and run through the parameters to build the element

In [41]:
preface = dict(
        practitioner = ("Doctor ","Practitioner"),
        service = ('Service: ',"HealthcareService"),
        location = ('Location: ',"Location"),
        )

def get_timeframe(timeframe):
    if timeframe == "today":
        start = str(date.today())
        end = str(date.today() + timedelta(days=1))
    elif timeframe == "tomorrow":
        start = str(date.today() + timedelta(days=1))
        end = str(date.today() + timedelta(days=2))
    else:
        start=df.admit_date.min()[:-9]
        end=df.admit_date.max()[:-9]       
    return((start,end))


def get_gc_code(code):
    cc = dict(
            text=code.capitalize(),
            coding=[dict(
                    system = "http://argonaut.example.org/list-types",
                    code = code,
                    display = code.capitalize()
                    )
                ]
            )
    return(cc)


def get_characteristic(parameter, v_type, value, start, end):
    print(v_type, value)
    period = dict(
                start = start,
                end = end,
            )
    my_gc = group.GroupCharacteristic(dict
                (
                exclude=False,
                code =  get_gc_code(parameter), #{"text":parameter},
                valueBoolean = True, #v_type default is boolean
                period = period,
                )
            )
    if v_type in ["reference"]:
        my_gc.valueBoolean= None
        my_gc.valueReference = fhirreference.FHIRReference(
                reference = f"{preface[parameter][1]}/{value}",
                 display = f"{preface[parameter][0]} {value.capitalize()}",
            )                                                    
    return(my_gc)
 
                                      
my_g.characteristic=[]
start,end = get_timeframe(timeframe)
                                      
for parameter in parameters: #loop through parameter variables get list of values
    print(parameter, timeframe)
    if parameter in ['service','location','practitioner']:
        v_type = 'reference'
        value = getattr(df, parameter).unique()[0] # all unique parameter values just get first one for now               
    else: # parameter in ['all_patients','discharged','active']:
        v_type = 'boolean'                                   
        value = True
    my_g.characteristic.append(get_characteristic(
        parameter=parameter, v_type = v_type, value = value, start=start, end=end
            ))
print(dumps(my_g.as_json(), indent =4))

service today
reference proctology
{
    "resourceType": "Group",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-2020-05-03T14:41:49.412645"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "all_patients",
    "managingEntity": {
        "display": "example managingEntity"
    },
    "characteristic": [
        {
            "code": {
                "coding": [
                    {
                        "system": "http://argonaut.example.org/list-types",
                        "code": "service",
                        "display": "Service"
                    }
                ],
                "text": "Service"
            },
            "valueReference": {
                "reference": "HealthcareService/proctology",
                "display": "Service:  Proctology"
            },
            "exclude": false,
            "period": {
                "start": "2020-05-03",
           

### Finally Add the Group Member from Excel

- for now use the business identifier instead of the FHIR Reference.
- get members based on characteristics defined above

In [42]:
def get_valueref(i):
    valueref =  i.valueReference.reference
    return(valueref.split('/')[1])


def get_filter(i):
    # filter by timeframe
    print(i.period.start.as_json(), i.period.end.as_json())
    tf_filter = (df.admit_date >= i.period.start.as_json()) & (df.admit_date <= i.period.end.as_json())
    #print(f'tf_filter =\n{tf_filter}')
    parameter = i.code.coding[0].code
    print(parameter)
    if parameter == "active":
        p_filter = (df.discharge_date >= i.period.end.as_json())
    if parameter == "discharged":
        p_filter = (df.discharge_date <= i.period.end.as_json())
    if parameter in ['practitioner','service','location']:
        print(get_valueref(i))
        p_filter = (df[parameter] == get_valueref(i))
    #print( f'p_filter =\n{p_filter}')
    #print(f'p_filter + tf_filter=\n{p_filter & tf_filter}')
    try:
        return(p_filter & tf_filter)
    except UnboundLocalError:
        return(tf_filter)

my_filter = True
for i in my_g.characteristic:
    my_filter = my_filter & get_filter(i)
    #print("my_filter:\n", my_filter)
df[my_filter]

2020-05-03 2020-05-04
service
proctology


,row,my_id,f_name,l_name,gender,b_date,is_deceased,is_active,phone,address1,city,state,zip,practitioner,location,service,admit_date,discharge_date
3,4,456,Daniel,Crimson,M,1996-05-23 00:00:00,False,True,5555555555,100 Yahome St,Napa,CA,94558,jones,St Helena,proctology,2020-05-03 00:00:00,2020-05-04 00:00:00
6,7,890,Greg,Orange,M,1982-05-01 00:00:00,False,True,5555555555,100 Elm St,Napa,CA,94558,smith,Queen,proctology,2020-05-03 00:00:00,2020-05-05 00:00:00


In [43]:
my_g.member =[]
for i in df[my_filter].itertuples(index=True):    
    my_member = group.GroupMember(
        dict(
            inactive= False,
            entity = dict(
                identifier = dict(
                        value = i.my_id,
                        system = "http://example.org",
                    ),
                display = f'Name = {i.f_name} {i.l_name} (MRN = {i.my_id})',
            ),
        )
    )
    
    my_g.member.append(my_member)

    
my_gj = dumps(my_g.as_json(), indent =4)

print(my_gj)

{
    "resourceType": "Group",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-2020-05-03T14:41:49.412645"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "all_patients",
    "managingEntity": {
        "display": "example managingEntity"
    },
    "characteristic": [
        {
            "code": {
                "coding": [
                    {
                        "system": "http://argonaut.example.org/list-types",
                        "code": "service",
                        "display": "Service"
                    }
                ],
                "text": "Service"
            },
            "valueReference": {
                "reference": "HealthcareService/proctology",
                "display": "Service:  Proctology"
            },
            "exclude": false,
            "period": {
                "start": "2020-05-03",
                "end": "2020-05-04"
          

   
### ======= Validate  =======
    

In [62]:
ref_server ='http://hapi.fhir.org/baseR4'
ref_server = 'http://test.fhir.org/r4'
ref_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
ref_server = 'https://vonk.fire.ly/R4'
headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
params = dict(
      )
print('validating ...')
r = post(f'{ref_server}/{my_g.resource_type}/$validate',
         params = params, headers = headers, data = my_gj)

try:
    display(HTML(
        '<h1>Validation output</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<h4>Response</h4><br />'
        f'{r.json()["text"]["div"]}'
        ))
    
except KeyError:
    display(HTML(
        '<h1>Validation output</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<h4>Response</h4><br />'
        f'<pre>{dumps(r.json(),indent=4)}</pre>'
        ))  

validating ...


### ======= Save/Load to ref FHIR Server ===========
  

In [63]:
print("......Posting to ref server ......")   
r = post(f'{ref_server}/{my_g.resource_type}',
          params = params, headers = headers, data = my_gj)
try:
    display(HTML(
        '<h1>Post Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))
except KeyError:
    display(HTML(
        '<h1>Post Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        #f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))   

......Posting to ref server ......


###  Todo:

- save to raw resources to file/ save ids as a dictionary.
- Add Narratives
- Fetch ids for patients to replace the identifiers
- create GET transactions as markdown and text

### Fetch Patient List based on Parameters

- docuement the transaction in markdwon

In [64]:
def pretty_print_request(req):
    """
    At this point it is completely built and ready
    to be fired; it is "prepared".

    However pay attention at the formatting used in 
    this function because it is programmed to be pretty 
    printed and may differ from the actual request.
    """
    req_headers = ('{}\n{}\r\n{}\r\n\r\n{}'.format(
        '-----------START-----------',
        req.method + ' ' + req.url,
        '\r\n'.join('{}: {}'.format(k, v) for k, v in req.headers.items()),
        req.body if req.body else '',
    ))
    return(req_headers)

params = {
    #"characteristic-value": "practitioner&Practitioner/smith",
    "characteristic-value": "service&HealthcareService/proctology",
    #"timeframe": "2020-05-01", # custom date SP for period
        }
print("......Fetching Patient List based on params......")    
r = get(f'{ref_server}/{my_g.resource_type}',
      params = params, headers = headers)

display(HTML(
        '<h1>Request</h1>'
        '<h3>Request Headers: </h3>'
        '<pre>'
        f'{pretty_print_request(r.request)}'
        '</pre>'
        '===============================================<br /><br /><br />'
        )
       )

try:
    display(HTML(
        '<h1>Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))
except KeyError:
    display(HTML(
        '<h1>Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        #f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))
r.json()

......Fetching Patient List based on params......


{'resourceType': 'OperationOutcome',
 'id': '96c8f5bb-a10c-44c0-b753-8527a92fb3fc',
 'meta': {'versionId': 'b1eac5a5-1ee3-4168-a6d4-3ddd0b351d73',
  'lastUpdated': '2020-05-03T22:05:33.534+00:00'},
 'issue': [{'severity': 'error',
   'code': 'invalid',
   'details': {'coding': [{'system': 'http://hl7.org/fhir/dotnet-api-operation-outcome',
      'code': '5012'}],
    'text': "Argument lacks the name/value separator '=' or has no value"}}]}